In [ ]:
from MetaWorld.utilsMW.trainer import ActiveCritic, ActiveCriticArgs
from stable_baselines3.common.torch_layers import CombinedExtractor
from MetaWorld.utilsMW.model_setup_obj import NetworkSetup
from stable_baselines3 import PPO, SAC
from stable_baselines3.common.vec_env import DummyVecEnv
import gym
from MetaWorld.searchTest.utils import MyEnv, ToyExpertModel, sample_expert_transitions, benchmark_policy, LearnWrapper, train_policy, LearnWrapper, parse_sampled_transitions, VecExtractor
import torch
import zipfile
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
import importlib
import numpy as np
from RlBaselines3Zoo import enjoy
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.ppo import MlpPolicy

from imitation.algorithms import bc
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper

from stable_baselines3.common.policies import MultiInputActorCriticPolicy, ActorCriticPolicy, BaseModel
from stable_baselines3.common.torch_layers import (
    BaseFeaturesExtractor,
    CombinedExtractor,
    FlattenExtractor,
    NatureCNN,
    create_mlp,
    get_actor_critic_arch,
)
from sb3_contrib.tqc.tqc import TQC
from sb3_contrib.tqc.policies import MultiInputPolicy
import numpy as np
import torch as th
import os

from MetaWorld.searchTest.utils import train_policy

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

model = enjoy.main(inpt_args="--algo tqc --env FetchPickAndPlace-v1 --folder /home/hendrik/Documents/master_project/Code/RlBaselines3Zoo/rl-trained-agents -n 300 --ret_model True")
def sample_expert_transitions():
    expert = model

    print("Sampling expert transitions.")
    rollouts = rollout.rollout(
        expert,
        DummyVecEnv([lambda: RolloutInfoWrapper(model.env.envs[0])]),
        rollout.make_sample_until(min_timesteps=None, min_episodes=100),
        unwrap=True,
        exclude_infos=False,
    )
    return rollout.flatten_trajectories(rollouts)

tp = '/home/hendrik/Documents/master_project/LokalData/ImitationLearning/transitions_rew_100'

#transitions = sample_expert_transitions()
#torch.save(transitions, tp)
transitions = torch.load('/home/hendrik/Documents/master_project/LokalData/ImitationLearning/transitions_rew_100')



In [ ]:
from MetaWorld.metaworld.policies.sawyer_pick_place_v2_policy import SawyerPickPlaceV2Policy
from metaworld.envs import ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE
from MetaWorld.utilsMW.makeTrainingData import make_policy_dict
from stable_baselines3.common.policies import ActorCriticPolicy
from stable_baselines3.common.on_policy_algorithm import OnPolicyAlgorithm
from typing import Union, Type, Optional
from stable_baselines3.common.type_aliases import GymEnv
from searchTest.utils import (
    ImitationLearningWrapper, 
    make_vec_env, 
    make_rollouts_vec, 
    DummyExtractor, 
    parse_sampled_transitions, 
    sample_expert_transitions,
    new_epoch,
    new_epoch_np
)

policy_dict = make_policy_dict()
gt_policy = policy_dict['pickplace']
pape = ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE[policy_dict['pickplace'][1]]
num_cpu = 1
ve1 = make_vec_env(num_cpu=num_cpu, env_constr=pape, epoch_len=100, seed=0)


IGTP = ImitationLearningWrapper(policy=gt_policy[0], env=ve1).predict

#transitions = sample_expert_transitions(IGTP)

In [25]:
ve1.reset()

array([[0.00615235, 0.6001898 , 0.19430117, 1.        , 0.0097627 ,
        0.67151893, 0.02      , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.00615235, 0.6001898 ,
        0.19430117, 1.        , 0.0097627 , 0.67151893, 0.02      ,
        0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.00897664, 0.84236548, 0.21147354]])

In [32]:
ve1.seed()

[[4203133778]]

In [30]:
ve1.reset()

array([[0.00615235, 0.6001898 , 0.19430117, 1.        , 0.0097627 ,
        0.67151893, 0.02      , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.00615235, 0.6001898 ,
        0.19430117, 1.        , 0.0097627 , 0.67151893, 0.02      ,
        0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.00897664, 0.84236548, 0.21147354]])

In [10]:
for i in range(99):
    obsv, rew, dones, info = ve1.step(np.array([[0,0,0,0]]))

In [17]:
obsv, rew, dones, info = ve1.step(np.array([[0,0,0,0]]))

In [19]:
dones

array([False])

In [ ]:
seeds = np.random.randint(0, 1e7, 1)
transitions = make_rollouts_vec(seeds=seeds, expert=IGTP, env_constr=pape, num_cpu=4, epoch_len=100)

In [ ]:
actions, observations, rewards = parse_sampled_transitions(transitions, new_epoch=new_epoch, extractor=DummyExtractor())

In [ ]:
actions.shape

In [ ]:
args_obj = ActiveCriticArgs()
args_obj.set_batchsize(32)
args_obj.set_data_path(path='/home/hendrik/Documents/master_project/LokalData/')
args_obj.set_device(device='cuda')
args_obj.set_feature_extractor(DummyExtractor())
args_obj.set_log_name('GT pickplace V2 100 Obsv')
args_obj.set_meta_optimizer_lr(1e-2)
args_obj.set_mlr(5e-5)
args_obj.set_network_setup(NetworkSetup())
args_obj.set_tboard(True)
args_obj.set_demonstrations(demonstrations=transitions)
args_obj.set_n_steps(n_steps=10000)
#args_obj.set_n_steps(n_steps=30)
args_obj.set_epoch_len(epoch_len=100)
args_obj.set_imitation_phase(True)
args_obj.set_weight_decay(1e-2)
args_obj.set_new_epoch(new_epoch=new_epoch)
args_obj.set_eval_epochs(epochs=50)
args_obj.set_opt_steps(20)
args_obj.set_complete_modulo(1)
args_obj.set_num_cpu(10)

pape = ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE[policy_dict['pickplace'][1]]
args_obj.set_env_factory(lambda num_cpu, env_constr, epoch_len, seed: make_vec_env(num_cpu=num_cpu, env_constr=env_constr, epoch_len=epoch_len, seed=seed))

ac = ActiveCritic(
    policy=None,
    env=pape,
    args_obj=args_obj,
    learning_rate=5e-5,
    extractor=DummyExtractor()
)

In [ ]:
#ac.network.loadNetworkFromFile(path='/home/hendrik/Documents/master_project/LokalData/Data/Model/GT pickplace V2 25/last/')

In [ ]:
train_policy(trainer=ac, 
learn_fct=ac.learn, 
val_env=ac.env, 
logname=args_obj.logname, 
path='/home/hendrik/Documents/master_project/LokalData/TransformerImitationLearning/',
n_epochs=1000,
n_steps=1,
eval_epochs=1,
step_fct=lambda i:i+1,
new_epoch=new_epoch,
extractor=DummyExtractor()
)

In [ ]:
a, b, c = ac.network.sample_new_episode(policy=ac.policy, env=dv1, episodes=5, add_data=False)

In [ ]:
r, t = benchmark_policy(
    policy=ac.policy, 
    path='/home/hendrik/Documents/master_project/LokalData/Data/Model/GT pickplace V2 25/',
    logname='reload test',
    eval_epochs=20,
    val_env=dv1,
    stepid=1,
    best_reward= -10,
    new_epoch=new_epoch,
    extractor=DummyExtractor(),
    save_model=False
    )

In [ ]:
trans = sample_expert_transitions(ac.policy)

In [ ]:
a,b,c = parse_sampled_transitions(transitions=trans, new_epoch=new_epoch, extractor=DummyExtractor())

In [ ]:
print(b.shape)

In [ ]:
print(c.shape)

In [ ]:
policy_kwargs = dict(net_arch=dict(pi=[64, 64], qf=[400, 300]))
# Create the agent
model = SAC("MlpPolicy", timelimit, verbose=1, policy_kwargs=policy_kwargs)
model.device

In [ ]:
bc_policy = ActorCriticPolicy(observation_space=timelimit.observation_space, action_space=timelimit.action_space, lr_schedule=lambda a:1, net_arch=[512,512,512])

In [ ]:
env = dv1
bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    demonstrations=transitions,
    policy=bc_policy,
    device='cuda'
)

In [ ]:
train_policy(
    trainer=bc_trainer,
    learn_fct=bc_trainer.train,
    val_env=dv1,
    logname='GT 10 V2 ActorCritic',
    path='/home/hendrik/Documents/master_project/LokalData/ImitationLearning/',
    n_epochs=1000,
    n_steps=50,
    eval_epochs=100,
    step_fct=lambda i: i+1,
    new_epoch=new_epoch,
    extractor=DummyExtractor()
)

In [ ]:
benchmark_policy(
    bc_policy, 
    path='/home/hendrik/Documents/master_project/LokalData/ImitationLearning/',
    logname='BC test gt pick place V2',
    eval_epochs=300,
    val_env=dv1,
    stepid=1,
    best_reward=-1,
    new_epoch=new_epoch,
    extractor=DummyExtractor(),
    save_model=False,
    )

In [ ]:
#ac.policy.load_state_dict(torch.load('/home/hendrik/Documents/master_project/LokalData/TransformerImitationLearning/First integrated test/best_modeltensor(-0.9200)'))

In [ ]:
train_policy(trainer=ac, 
learn_fct=ac.learn, 
val_env=ac.env, 
logname='GT pick place V2', 
path='/home/hendrik/Documents/master_project/LokalData/TransformerImitationLearning/',
n_epochs=1000,
n_steps=1,
eval_epochs=1,
step_fct=lambda i:i+1,
new_epoch=new_epoch,
extractor=DummyExtractor()
)

In [ ]:
ac.learn(n_epochs=1)

In [ ]:
ac.network.runValidation(quick=False, pnt=True, epoch=1, save=False, complete=True)